<a href="https://colab.research.google.com/github/HidemaruOwO/Google-Colab-Anythings-V3-and-realcugan-ncnn-vulkan/blob/master/Anything_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
# パッケージのインストール
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install pytorch_lightning tensorboard==2.9.1 omegaconf einops taming-transformers==0.0.1 clip transformers kornia test-tube
!pip install diffusers invisible-watermark

In [ ]:
# StableDiffusion のインストール
!git clone https://github.com/CompVis/stable-diffusion.git
%cd stable-diffusion
!pip install -e .
%mkdir outputs

In [ ]:
# Anythingv3.0 のモデルデータを取得
!wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned.ckpt

In [ ]:
# セーフフィルタのチェックをスキップした txt2img.py を txt2img2.py として保存
!wget https://gist.githubusercontent.com/uakihir0/bbb1bd8a4480e2bab07726ca0e744f91/raw/3e6f43fa5c112e512006cc9e33e932f6c6539d94/txt2img.py -O scripts/txt2img2.py

In [ ]:
# realcugan ncnn vulkanの追加
!git clone https://github.com/HidemaruOwO/realcugan-ncnn-vulkan-simple
!mv realcugan-ncnn-vulkan-simple/* .

In [ ]:
# realcugan ncnn vulkanの環境構築
!bash setup.sh
!mv convert.sh convert.sh.tmp
!cat convert.sh.tmp | sed -s "s;baseimg;outputs/samples;" >> convert.sh
!chmod 744 convert.sh
!apt install vulkan-utils libvulkan-dev

In [ ]:
# realcugan-ncnn-vulkanが落とせなかったら直接落とす
!wget https://github.com/nihui/realcugan-ncnn-vulkan/releases/download/20220728/realcugan-ncnn-vulkan-20220728-ubuntu.zip
!unzip realcugan-ncnn-vulkan-20220728-ubuntu.zip
!mv realcugan-ncnn-vulkan-20220728-ubuntu/* bin

In [ ]:
# イラストの生成
# promptに英単語および英文を羅列させる
!python scripts/txt2img2.py \
    --H 704 \
    --W 448 \
    --plms \
    --ckpt ./Anything-V3.0-pruned.ckpt \
    --skip_grid \
    --n_samples 1  \
    --n_iter 1 \
    --outdir outputs \
    --ddim_steps 100 \
    --prompt "JK high school student black haer eyes cute heart in eyes sleepy lie prone on one's desk"

In [ ]:
# 生成した画像のアップスケーリング
# 第一引数: 拡大率
# 第二引数: ノイズ除去の強度
!bash convert.sh 3 3